# Preparing the receptor

In [2]:
from openeye import oechem, oedocking
import sys, os

In [4]:
ifs = oechem.oemolistream()
ofs = oechem.oemolostream()

I need to deal with file paths better. Not totally sure how to do this in python

In [7]:
com = oechem.OEGraphMol()
if ifs.open("input/com_AXI.pdb"):
    oechem.OEReadPDBFile(ifs,com)
    ifs.close()

In [8]:
# Find the connected components of the system, remove the ligand
oechem.OEDetermineConnectivity(com)
nparts, connect = oechem.OEDetermineComponents(com)

print('Number of connected components:', nparts)

Number of connected components: 2


This will break if the last connected component isn't the ligand, or if the protein has multiple chains.

These could be fixed...


In [9]:
# Separate the ligand                                                           
pred = oechem.OEPartPredAtom(connect)
pred.SelectPart(nparts)
lig = oechem.OEGraphMol()
oechem.OESubsetMol(lig, com, pred)

if ofs.open("input/original_lig.mol2"):
    oechem.OEWriteMolecule(ofs,lig)
    ofs.close()

In [10]:
# Separate the protein                                        
del pred
pred = oechem.OEPartPredAtom(connect)
pred.SelectPart(1)
prot = oechem.OEGraphMol()
oechem.OESubsetMol(prot, com, pred)

if ofs.open("input/apo_protein.mol2"):
    oechem.OEWriteMolecule(ofs,prot)
    ofs.close()

In [12]:
# Now iterate over these indices to create a box for docking                    
x_min = y_min = z_min = float('inf')
x_max = y_max = z_max = -float('inf')
crd = lig.GetCoords()
for atm in crd:
    x,y,z = crd[atm]
    if x < x_min:
        x_min = x
    if y < y_min:
        y_min = y
    if z < z_min:
        z_min = z
    if x > x_max:
        x_max = x
    if y > y_max:
        y_max = y
    if z > z_max:
        z_max = z

In [13]:
# Pad the box to be slightly larger than the existing molecule
# My choice of padding by 2 Angstrom is arbitrary.
x_min -= 2
y_min -= 2
z_min -= 2
x_max += 2
y_max += 2
z_max += 2

In [15]:
# Now prepare the receptor
receptor = oechem.OEGraphMol()
box = oedocking.OEBox(x_max, y_max, z_max, x_min, y_min, z_min)
oedocking.OEMakeReceptor(receptor, prot, box)

True

In [19]:
oedocking.OEWriteReceptorFile(receptor,"input/receptor.oeb")

True